In [4]:
import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import applyG_block_state, get_perms

import numpy as np
import quimb.tensor as qtn
import quimb
from quimb.tensor.tensor_2d_tebd import TEBD2D, LocalHam2D
import scipy
import h5py
import qib
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply

import time
import tracemalloc
tracemalloc.start()

t = 0.125
Lx, Ly = (6, 6)
L = Lx*Ly
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)

perms_v, perms_h = (
    [[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
    [1, 2, 3, 4, 5, 0, 7, 8, 9, 10, 11, 6, 13, 14, 15, 16, 17, 12, 19, 20, 21, 22, 23, 18, 25, 26, 27, 28, 29, 24, 31, 32, 33, 34, 35, 30]],
    [[0, 6, 12, 18, 24, 30, 1, 7, 13, 19, 25, 31, 2, 8, 14, 20, 26, 32, 3, 9, 15, 21, 27, 33, 4, 10, 16, 22, 28, 34, 5, 11, 17, 23, 29, 35], 
    [6, 12, 18, 24, 30, 0, 7, 13, 19, 25, 31, 1, 8, 14, 20, 26, 32, 2, 9, 15, 21, 27, 33, 3, 10, 16, 22, 28, 34, 4, 11, 17, 23, 29, 35, 5]]
)

#perms_v, perms_h = ([[0, 4, 1, 5, 2, 6, 3, 7, 8, 12, 9, 13, 10, 14, 11, 15],
#  [4, 8, 5, 9, 6, 10, 7, 11, 12, 0, 13, 1, 14, 2, 15, 3]],
# [[0, 1, 4, 5, 8, 9, 12, 13, 2, 3, 6, 7, 10, 11, 14, 15],
#  [1, 2, 5, 6, 9, 10, 13, 14, 3, 0, 7, 4, 11, 8, 15, 12]])
#hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()

perms_extended = [[perms_v[0]]] + [perms_v]*3 + [[perms_v[0]], [perms_h[0]]] +\
                    [perms_h]*3 + [[perms_h[0]]]
perms_ext_reduced = [perms_v]*3  + [perms_h]*3
control_layers = [0, 4, 5, 9] 			# 4 control layers
Vlist = []
with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.125_layers10_niter10_rS1_2hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]

map_ = {i: (i//Ly, i%Lx) for i in range(L)}

peps = qtn.PEPS.rand(Lx, Ly, bond_dim=1, phys_dim=2, cyclic=True)
peps = peps / peps.norm()
peps_E = peps.copy()
peps_T = peps.copy()
peps_C = peps.copy()


BD = 5
nsteps = 2
peps_E = trotter(peps_E.copy(), t, L, Lx, Ly, J, g, perms_v, perms_h,
                     dt=t/nsteps, max_bond_dim=BD, lower_max_bond_dim=BD, trotter_order=2)
peps_T = trotter(peps_T.copy(), t, L, Lx, Ly, J, g, perms_v, perms_h,
                     dt=t/nsteps, max_bond_dim=BD, lower_max_bond_dim=BD, trotter_order=1)
peps_aE = ccU(peps_C.copy(), Vlist, perms_extended, control_layers, dagger=False,
                 max_bond_dim=BD, lower_max_bond_dim=BD)


Trotter steps  2
Trotter steps  2


In [ ]:
chi_norm = 256  # "chi" – boundary MPS bond dimension
for state in (peps_E, peps_T, peps_aE):
    # normalize in-place using approximate boundary contraction
    state.normalize(
        max_bond=chi_norm,
        cutoff=1e-10,
        mode="mps",
        layer_tags=("KET", "BRA"),
        inplace=True,
    )
ov_tn = peps_E.make_overlap(
    peps_aE,
    layer_tags=("KET", "BRA"),
)
chi_overlap = 256  # max internal bond during compressed contraction
overlap_approx = ov_tn.contract_compressed(
    optimize="auto-hq",  # preset strategy name understood via cotengra
    max_bond=chi_overlap,
    cutoff=1e-10,
    # leave strip_exponent=False (default) so we just get a scalar back
)

np.abs(overlap_approx)

/opt/anaconda3/envs/qc/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:36: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead. `kahypar` is highly recommended for the best quality contraction paths.
  warnings.warn(


In [121]:
chi_norm = 256  # "chi" – boundary MPS bond dimension
for state in (peps_E, peps_T, peps_aE):
    # normalize in-place using approximate boundary contraction
    state.normalize(
        max_bond=chi_norm,
        cutoff=1e-10,
        mode="mps",
        layer_tags=("KET", "BRA"),
        inplace=True,
    )
ov_tn = peps_E.make_overlap(
    peps_T,
    layer_tags=("KET", "BRA"),
)
chi_overlap = 256  # max internal bond during compressed contraction
overlap_approx = ov_tn.contract_compressed(
    optimize="auto-hq",  # preset strategy name understood via cotengra
    max_bond=chi_overlap,
    cutoff=1e-10,
    # leave strip_exponent=False (default) so we just get a scalar back
)
np.abs(overlap_approx)

/opt/anaconda3/envs/qc/lib/python3.10/site-packages/quimb/tensor/tensor_core.py:8584: UserWarning: The contraction tree is not a compressed one, this may be very inefficient.
  warnings.warn(


0.9805823435507475

In [116]:
from qiskit.quantum_info import state_fidelity

sv1 =  peps_E.to_dense()
sv1 /= np.linalg.norm(sv1)

sv2 = peps.to_dense()
sv2 /= np.linalg.norm(sv2)

state_fidelity(sv1, scipy.sparse.linalg.expm_multiply(-1j * t * hamil, sv2) )

0.9978077305187916

In [2]:
import rqcopt as oc
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
I2 = np.eye(2)

def trotter(peps, t, L, Lx, Ly, J, g, perms_v, perms_h, dag=False, max_bond_dim=5, 
            dt=0.1, trotter_order=2, treshold=10, lower_max_bond_dim=4):
    nsteps = np.abs(int(np.ceil(t/dt)))
    print("Trotter steps ", nsteps)
    dt = t/nsteps
    if trotter_order > 1:
        indices = oc.SplittingMethod.suzuki(2, int(np.log(trotter_order)/np.log(2))).indices
        coeffs = oc.SplittingMethod.suzuki(2, int(np.log(trotter_order)/np.log(2))).coeffs
    else:
        indices = [0, 1]
        coeffs = [1, 1]
    
    hloc1 = g*(np.kron(X, I2)+np.kron(I2, X))/4
    hloc2 = J*np.kron(Z, Z)
    hlocs = (hloc1, hloc2)
    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(-1j*c*dt*hlocs[i])

    for n in range(nsteps):
        for layer, V in enumerate(Vlist_start):
            i = n*len(Vlist_start)+layer
            for perm in perms_h:
                ordering = {(map_[perm[2*j]], map_[perm[2*j+1]]): V for j in range(L//2)}
                start = time.time()
                t = TEBD2D(peps, ham=LocalHam2D(Lx, Ly, ordering, cyclic=True),
                    tau=-1, D=max_bond_dim if i<treshold else lower_max_bond_dim, chi=1)
                t.sweep(tau=-1)
                peps = t.state
                
            for perm in perms_v:
                ordering = {(map_[perm[2*j]], map_[perm[2*j+1]]): V for j in range(L//2)}
                start = time.time()
                t = TEBD2D(peps, ham=LocalHam2D(Lx, Ly, ordering, cyclic=True),
                    tau=-1, D=max_bond_dim if i<treshold else lower_max_bond_dim, chi=1)
                t.sweep(tau=-1)
                peps = t.state
            #with open(f"trotter_PEPS_log{Lx}{Ly}.txt", "a") as file:
            #    file.write(f"Time step {n}/{nsteps}, layer {layer}/{len(Vlist_start)} applied \n")
            #print(f"Time step {n}/{nsteps}, layer {layer}/{len(Vlist_start)} applied")
    #print(f"Norm. Trotter")
    #peps.normalize()
    #peps = peps / peps.norm()
    #print(f"norm finished Trotter")
    return peps

In [3]:
def ccU(peps, Vlist, perms_extended, control_layers, dagger=False, max_bond_dim=10, lower_max_bond_dim=4, treshold=10):
    for i, V in enumerate(Vlist):
        if dagger or i not in control_layers:
            perms = perms_extended[i]
            for perm in perms:
                ordering = {(map_[perm[2*j]], map_[perm[2*j+1]]): scipy.linalg.logm(V) for j in range(L//2)}
                start = time.time()
                t = TEBD2D(peps, ham=LocalHam2D(Lx, Ly, ordering, cyclic=True),
                    tau=-1, D=max_bond_dim if i<treshold else lower_max_bond_dim, chi=1)
                t.sweep(tau=-1)
                peps = t.state
                #peps /= peps.norm()
                #print(f"Step {i} took {time.time() - start:.2f} seconds")
                #print("Peak memory:", tracemalloc.get_traced_memory())
    #print(f"Normalize ccU")
    #peps = peps / peps.norm()
    #print(f"Normalization of ccU finished")
    return peps

In [ ]:
"""Vlist = []
with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_rS1_niter15_3hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]
control_layers = [0, 4, 5, 9, 10, 14]
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_niter20_rS5_DECOMPOSE_n9_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]"""

#peps_aE2 = ccU(peps_aE.copy(), Vlist, perms_extended, control_layers, dagger=True, 
#                 max_bond_dim=4, lower_max_bond_dim=4, treshold=10)

#peps_trotters[.25] = peps_T.copy()
#peps_ccUs[.25] = peps_aE.copy()
#peps_exacts[.25] = peps_E.copy()

#sv = peps.to_dense()[:, 0]
#sv = expm_multiply(1j * 0.25 * hamil, sv)
#peps_exact_05 = trotter(peps_E.copy(), -0.25, L, Lx, Ly, J, g, perms_v, perms_h, dt=0.25/10, max_bond_dim=3, lower_max_bond_dim=3, treshold=10)
#peps_trotter_05 = trotter(peps_T.copy(), -0.25, L, Lx, Ly, J, g, perms_v, perms_h, dt=0.25/2, max_bond_dim=3, lower_max_bond_dim=3, treshold=10)
#f = quimb.fidelity(peps_trotter.to_dense()[:, 0], sv)
#print("Trotter Fidelity:", f)  # Should be ≈1
#peps_ccU_025 = ccU(peps_C.copy(), Vlist, perms_extended, control_layers, dagger=True, max_bond_dim=3, lower_max_bond_dim=3, treshold=10)
#f = quimb.fidelity(peps_ccU.to_dense()[:, 0], sv)
#print("ccU Fidelity after identity:", f)  # Should be ≈1
#np.linalg.norm(peps_ccU.overlap(peps_trotter))**2
